In [ ]:
!pip install google-generativeai python-dotenv SpeechRecognition gradio gtts

### Import library

In [ ]:
import os
import json
import subprocess
import webbrowser
import time
from dotenv import load_dotenv, find_dotenv, set_key
from openai import OpenAI
import gradio as gr

# --- Function to Create Assistant and Save ID ---
def create_assistant(client):
    """Creates a new assistant and saves its ID to the .env file."""
    assistant = client.beta.assistants.create(
        name="System Commander",
        instructions="You are an AI system assistant that can open apps, search the web, read files, and run scripts. Always confirm before running anything destructive.",
        tools=[
            {"type": "function", "function": {"name": "open_application", "description": "Open system applications", "parameters": {"type": "object", "properties": {"app_name": {"type": "string", "description": "Name of the app to open"}}, "required": ["app_name"]}}},
            {"type": "function", "function": {"name": "search_web", "description": "Search something on the internet", "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}}},
            {"type": "function", "function": {"name": "read_file", "description": "Read contents of a text file", "parameters": {"type": "object", "properties": {"file_path": {"type": "string", "description": "Path to the text file"}}, "required": ["file_path"]}}},
            {"type": "function", "function": {"name": "run_script", "description": "Execute a script (bash or Python)", "parameters": {"type": "object", "properties": {"script_path": {"type": "string", "description": "Script path (must be inside ./scripts/)"}}, "required": ["script_path"]}}}
        ],
        model="gpt-4o"
    )
    
    dotenv_path = find_dotenv()
    if not dotenv_path:
        # Create a .env file if it doesn't exist
        with open(".env", "w") as f:
            pass
        dotenv_path = find_dotenv()

    set_key(dotenv_path, "ASSISTANT_ID", assistant.id)
    print(f"✅ New assistant created and ID saved to .env: {assistant.id}")
    return assistant.id

# --- Environment Variable and Client Setup ---
def setup_environment():
    """Loads environment variables, creates an assistant if needed, and returns the client and ID."""
    load_dotenv()
    
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key or not api_key.startswith("sk-"):
        raise gr.Error("❌ Invalid or missing OpenAI API Key. Please check your .env file.")
    
    client = OpenAI(api_key=api_key)
    
    assistant_id = os.getenv("ASSISTANT_ID")
    if not assistant_id:
        print("Assistant ID not found in .env. Creating a new one...")
        assistant_id = create_assistant(client)
        
    print(f"Using Assistant ID: {assistant_id}")
    return client, assistant_id

try:
    client, ASSISTANT_ID = setup_environment()
except gr.Error as e:
    raise e # Re-raise Gradio errors to be displayed in the UI

# --- Tool Functions (No changes needed here) ---
def open_application(app_name: str) -> str:
    """Opens a system application."""
    try:
        if os.name == 'nt': subprocess.Popen(f'start {app_name}', shell=True)
        else: subprocess.Popen(['open' if os.sys.platform == 'darwin' else 'xdg-open', app_name])
        return f"✅ Successfully started {app_name}."
    except Exception as e: return f"❌ Error opening application: {str(e)}"

def search_web(query: str) -> str:
    """Searches the web using the default browser."""
    try:
        webbrowser.open(f"https://www.google.com/search?q={query}")
        return f"🔍 Searching for: '{query}'"
    except Exception as e: return f"❌ Error searching web: {str(e)}"

def read_file(file_path: str) -> str:
    """Reads the content of a file."""
    try:
        if not os.path.exists(file_path): return "❌ File not found."
        with open(file_path, 'r', encoding='utf-8') as f: return f.read()[:5000]
    except Exception as e: return f"❌ Error reading file: {str(e)}"

def run_script(script_path: str) -> str:
    """Runs a script from an allowed directory."""
    try:
        allowed_paths = ["./scripts/", "/scripts/"]
        if not any(os.path.abspath(script_path).startswith(os.path.abspath(p)) for p in allowed_paths):
            return "❌ SecurityError: Access to this path is denied."
        interpreter = "python" if script_path.endswith(".py") else "bash"
        result = subprocess.run([interpreter, script_path], capture_output=True, text=True, timeout=30, check=True)
        return result.stdout.strip() or "🟢 Script executed successfully with no output."
    except FileNotFoundError: return f"❌ Error: Script '{script_path}' not found."
    except subprocess.CalledProcessError as e: return f"❌ Script failed:\n{e.stderr}"
    except Exception as e: return f"❌ Unexpected error running script: {str(e)}"

# --- Tool Call Handler ---
def execute_tool_call(call):
    """Dynamically executes the appropriate tool function."""
    func_name = call.function.name
    args = json.loads(call.function.arguments)
    tool_functions = {"open_application": open_application, "search_web": search_web, "read_file": read_file, "run_script": run_script}
    if func_name in tool_functions: return tool_functions[func_name](**args)
    else: return f"❌ Error: Unknown function '{func_name}'."

# --- Main Chat Logic ---
def process_chat(user_input, history, thread_id):
    """Handles the main chat interaction, thread management, and assistant responses."""
    if not user_input.strip(): 
        return history, thread_id, ""

    try:
        if thread_id is None: 
            thread = client.beta.threads.create()
            thread_id = thread.id
            
        client.beta.threads.messages.create(thread_id=thread_id, role="user", content=user_input)
        run = client.beta.threads.runs.create(thread_id=thread_id, assistant_id=ASSISTANT_ID)

        start_time = time.time()
        while time.time() - start_time < 60:
            run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run_status.status == 'completed': break
            if run_status.status == 'requires_action':
                tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                tool_outputs = [{"tool_call_id": call.id, "output": str(execute_tool_call(call))} for call in tool_calls]
                client.beta.threads.runs.submit_tool_outputs(thread_id=thread_id, run_id=run.id, tool_outputs=tool_outputs)
            if run_status.status in ['failed', 'cancelled', 'expired']:
                error_message = f"⚠️ Run failed: `{run_status.status}`"
                if run_status.last_error: error_message += f"\n\n**Details:**\n{run_status.last_error.message}"
                history.append((user_input, error_message))
                return history, thread_id, ""
            time.sleep(1)

        if run_status.status != 'completed':
            history.append((user_input, "⏱️ Timeout: Assistant took too long."))
            return history, thread_id, ""

        messages = client.beta.threads.messages.list(thread_id=thread_id)
        assistant_response = "⚠️ No message returned."
        for msg in reversed(messages.data):
            if msg.run_id == run.id and msg.role == 'assistant' and msg.content[0].type == 'text':
                assistant_response = msg.content[0].text.value
                break
        
        history.append((user_input, assistant_response))
        return history, thread_id, ""

    except Exception as e:
        error_text = f"An unexpected application error occurred: {str(e)}"
        history.append((user_input, error_text))
        return history, thread_id, ""

# --- Gradio Interface Setup ---
with gr.Blocks(theme=gr.themes.Soft(), title="System Commander AI") as demo:
    gr.Markdown("# 🧠 System Commander AI")
    gr.Markdown("Ask your assistant to open apps, search the web, run scripts, or read files!")
    
    # State to store the conversation thread ID for the session
    thread_state = gr.State(None)

    chatbot = gr.Chatbot(label="Conversation", height=600)
    
    with gr.Row():
        textbox = gr.Textbox(
            show_label=False,
            placeholder="Type something like: open notepad OR search for AI tools",
            scale=4,
            container=False
        )
        submit_btn = gr.Button("Send", variant="primary", scale=1)

    # Define the click and submit events to handle the interaction
    submit_btn.click(
        fn=process_chat,
        inputs=[textbox, chatbot, thread_state],
        outputs=[chatbot, thread_state, textbox]
    )
    textbox.submit(
        fn=process_chat,
        inputs=[textbox, chatbot, thread_state],
        outputs=[chatbot, thread_state, textbox]
    )

if __name__ == "__main__":
    demo.launch()


In [ ]:
import os
import json
import subprocess
import webbrowser
import time
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Load environment variables
load_dotenv()
OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY or not OPENAI_API_KEY.startswith("sk-"):
    raise ValueError("❌ Invalid or missing OpenAI API Key in .env file")

# Initialize OpenAI Client
client = OpenAI(api_key=OPENAI_API_KEY)

# --------- TOOL FUNCTIONS ---------
def open_application(app_name: str) -> str:
    try:
        if os.name == 'nt':
            subprocess.Popen(f'start {app_name}', shell=True)
        elif os.name == 'posix':
            subprocess.Popen(['open' if 'darwin' in os.sys.platform else 'xdg-open', app_name])
        return f"✅ Opened {app_name}"
    except Exception as e:
        return f"❌ Error opening app: {str(e)}"

def search_web(query: str) -> str:
    try:
        webbrowser.open(f"https://www.google.com/search?q={query}")
        return f"🔍 Searched for: {query}"
    except Exception as e:
        return f"❌ Error searching web: {str(e)}"

def read_file(file_path: str) -> str:
    try:
        if not os.path.exists(file_path):
            return "❌ File not found"
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()[:5000]
    except Exception as e:
        return f"❌ Error reading file: {str(e)}"

def run_script(script_path: str) -> str:
    try:
        allowed_paths = ["./scripts/", "/scripts/"]
        if not any(script_path.startswith(path) for path in allowed_paths):
            return "❌ Access denied to path"
        result = subprocess.run(script_path, shell=True, capture_output=True, text=True, timeout=30)
        return result.stdout or "🟢 Script executed"
    except Exception as e:
        return f"❌ Error running script: {str(e)}"

# --------- ASSISTANT SETUP (Only First Time) ---------
def setup_assistant():
    assistant = client.beta.assistants.create(
        name="System Commander",
        instructions="You are an AI system assistant that can open apps, search the web, read files, and run scripts. Always confirm before running anything destructive.",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "open_application",
                    "description": "Open system applications",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "app_name": {"type": "string", "description": "Name of the app to open"}
                        },
                        "required": ["app_name"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_web",
                    "description": "Search something on the internet",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Search query"}
                        },
                        "required": ["query"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "read_file",
                    "description": "Read contents of a text file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {"type": "string", "description": "Path to the text file"}
                        },
                        "required": ["file_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "run_script",
                    "description": "Execute a script (bash or Python)",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "script_path": {"type": "string", "description": "Script path (must be inside ./scripts/)"}
                        },
                        "required": ["script_path"]
                    }
                }
            }
        ],
        model="gpt-4o"
    )
    print(f"✅ Assistant Created: {assistant.id}")
    return assistant.id

# Uncomment this only the first time
# ASSISTANT_ID = setup_assistant()
# Or paste your assistant ID below if already created
ASSISTANT_ID = "asst_cPw2PUu2qzrHLF2E8itq8ESU"

# --------- TOOL CALL HANDLER ---------
def execute_tool_call(call):
    func_name = call.function.name
    args = json.loads(call.function.arguments)

    if func_name == "open_application":
        return open_application(args["app_name"])
    elif func_name == "search_web":
        return search_web(args["query"])
    elif func_name == "read_file":
        return read_file(args["file_path"])
    elif func_name == "run_script":
        return run_script(args["script_path"])
    else:
        return "❌ Unknown function"

# --------- GRADIO INTERFACE ---------
chat_thread = client.beta.threads.create()

def gradio_chat(user_input):
    try:
        client.beta.threads.messages.create(
            thread_id=chat_thread.id,
            role="user",
            content=user_input
        )

        run = client.beta.threads.runs.create(
            thread_id=chat_thread.id,
            assistant_id=ASSISTANT_ID
        )

        max_attempts = 30
        attempts = 0

        while attempts < max_attempts:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=chat_thread.id,
                run_id=run.id
            )

            if run_status.status == "completed":
                break

            if run_status.status == "requires_action":
                tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []

                for call in tool_calls:
                    output = execute_tool_call(call)
                    tool_outputs.append({
                        "tool_call_id": call.id,
                        "output": output
                    })

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=chat_thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

            time.sleep(10)
            attempts += 1
        else:
            return "⏱️ Timeout waiting for assistant."

        messages = client.beta.threads.messages.list(
            thread_id=chat_thread.id,
            order="asc"
        )

        for msg in reversed(messages.data):
            if msg.role == "assistant":
                for content in msg.content:
                    if content.type == "text":
                        return content.text.value

        return "⚠️ Assistant completed, but no message was returned."

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# --------- LAUNCH GRADIO ---------
gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(lines=2, placeholder="Type something like: open notepad OR search AI tools"),
    outputs=gr.Textbox(label="Assistant"),
    title="🧠 System Commander AI",
    description="Ask your assistant to open apps, search web, run scripts, or read files!"
).launch()


C:\Users\mohit\AppData\Local\Temp\ipykernel_23072\4047494791.py:147: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  chat_thread = client.beta.threads.create()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


C:\Users\mohit\AppData\Local\Temp\ipykernel_23072\2782777661.py:134: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(thread_id=thread_id, role="user", content=user_input)
C:\Users\mohit\AppData\Local\Temp\ipykernel_23072\2782777661.py:135: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(thread_id=thread_id, assistant_id=ASSISTANT_ID)


In [ ]:
import os
import json
import subprocess
import webbrowser
import time
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# System Tools Implementation
def open_application(app_name: str) -> str:
    """Open applications cross-platform"""
    try:
        if os.name == 'nt':  # Windows
            os.startfile(app_name)
        elif os.name == 'posix':  # macOS/Linux
            subprocess.Popen(['open' if 'darwin' in os.sys.platform else 'xdg-open', app_name])
        return f"✅ Opened {app_name}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def search_web(query: str) -> str:
    """Perform web search"""
    try:
        webbrowser.open(f"https://www.google.com/search?q={query}")
        return f"🔍 Searched for: {query}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def read_file(file_path: str) -> str:
    """Read file contents"""
    try:
        if not os.path.exists(file_path):
            return "❌ File not found"
        with open(file_path, 'r') as f:
            return f.read()[:5000]  # Limit to first 5000 chars
    except Exception as e:
        return f"❌ Error: {str(e)}"

def run_script(script_path: str) -> str:
    """Execute scripts safely"""
    try:
        # Security check - validate path
        allowed_paths = ["/scripts/", "./scripts/"]
        if not any(path in script_path for path in allowed_paths):
            return "❌ Access denied to path"

        result = subprocess.run(
            script_path,
            shell=True,
            capture_output=True,
            text=True,
            timeout=30  # Prevent long-running scripts
        )
        return result.stdout or "🟢 Script executed"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Initialize OpenAI Client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Create Assistant (run once then save ID)
def setup_assistant():
    assistant = client.beta.assistants.create(
        name="System Commander",
        instructions="You are an AI system assistant that can execute computer commands. Confirm dangerous actions with user. Never run destructive commands.",
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "open_application",
                    "description": "Open applications like browsers, editors, or system apps",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "app_name": {"type": "string", "description": "Application name e.g. 'chrome', 'spotify'"}
                        },
                        "required": ["app_name"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_web",
                    "description": "Perform web searches",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Search keywords"}
                        },
                        "required": ["query"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "read_file",
                    "description": "Read text files",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {"type": "string", "description": "Relative or absolute file path"}
                        },
                        "required": ["file_path"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "run_script",
                    "description": "Execute scripts or commands",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "script_path": {"type": "string", "description": "Path to script or command"}
                        },
                        "required": ["script_path"]
                    }
                }
            }
        ],
        model="gpt-4o-mini"
    )
    print(f"Assistant created! ID: {assistant.id}")
    return assistant.id

# Uncomment to create new assistant (first run)
ASSISTANT_ID = setup_assistant()
# Save this ID for future use
#ASSISTANT_ID = "asst_ymyT94bTVeCX4ym7tQN99Lsg"  # Replace with your actual ID

# Tool Execution Handler
def execute_tool_call(tool_call):
    func_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"⚙️ Executing: {func_name}({args})")

    if func_name == "open_application":
        return open_application(args["app_name"])
    elif func_name == "search_web":
        return search_web(args["query"])
    elif func_name == "read_file":
        return read_file(args["file_path"])
    elif func_name == "run_script":
        return run_script(args["script_path"])
    else:
        return "⚠️ Unknown command"

# Main Chat Loop
def main():
    thread = client.beta.threads.create()

    print("🚀 System Assistant Ready (type 'exit' to quit)")

    while True:
        try:
            user_input = input("\n👤 You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            # Add user message to thread
            client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=user_input
            )

            # Run assistant
            run = client.beta.threads.runs.create(
                thread_id=thread.id,
                assistant_id=ASSISTANT_ID
            )

            # Process tool calls
            while True:
                run_status = client.beta.threads.runs.retrieve(
                    thread_id=thread.id,
                    run_id=run.id
                )

                if run_status.status == "completed":
                    break

                if run_status.status == "requires_action":
                    tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                    tool_outputs = []

                    for call in tool_calls:
                        output = execute_tool_call(call)
                        tool_outputs.append({
                            "tool_call_id": call.id,
                            "output": output
                        })

                    # Submit tool outputs
                    run = client.beta.threads.runs.submit_tool_outputs(
                        thread_id=thread.id,
                        run_id=run.id,
                        tool_outputs=tool_outputs
                    )

                time.sleep(1)

            # Get final response
            messages = client.beta.threads.messages.list(
                thread_id=thread.id,
                order="asc"
            )

            # Print assistant responses
            for msg in messages.data:
                if msg.role == "assistant":
                    for content in msg.content:
                        if content.type == "text":
                            print(f"\n🤖 Assistant: {content.text.value}")

        except Exception as e:
            print(f"⚠️ Error: {str(e)}")

#if __name__ == "__main__":
#    main()
import gradio as gr

# Prepare thread globally
chat_thread = client.beta.threads.create()

def gradio_chat(user_input):
    try:
        client.beta.threads.messages.create(
            thread_id=chat_thread.id,
            role="user",
            content=user_input
        )

        run = client.beta.threads.runs.create(
            thread_id=chat_thread.id,
            assistant_id=ASSISTANT_ID
        )

        # Add timeout mechanism (max 30 seconds)
        max_attempts = 30
        attempts = 0

        while attempts < max_attempts:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=chat_thread.id,
                run_id=run.id
            )

            if run_status.status == "completed":
                break

            if run_status.status == "requires_action":
                tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []

                for call in tool_calls:
                    output = execute_tool_call(call)
                    tool_outputs.append({
                        "tool_call_id": call.id,
                        "output": output
                    })

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=chat_thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

            time.sleep(1)
            attempts += 1
        else:
            return "⏱️ Timeout: Assistant took too long to respond."

        # Get response
        messages = client.beta.threads.messages.list(
            thread_id=chat_thread.id,
            order="asc"
        )

        for msg in reversed(messages.data):  # Check latest first
            if msg.role == "assistant":
                for content in msg.content:
                    if content.type == "text":
                        return content.text.value
            return" ⚠️ Assistant responded but no message"
    
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


# Build Gradio UI
gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(lines=2, placeholder="Type a command for your system assistant..."),
    outputs=gr.Textbox(label="Assistant"),
    title="🧠 System Commander AI",
    description="Ask your assistant to open apps, search the web, run scripts, or read files!"
).launch()


Assistant created! ID: asst_PXRtbG4Fyh83RtvYDp0uXRAw


C:\Users\mohit\AppData\Local\Temp\ipykernel_26056\3763315997.py:231: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  chat_thread = client.beta.threads.create()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


### experiment


In [7]:
import os
import json
import subprocess
import webbrowser
import queue
import threading
import speech_recognition as sr
import pyaudio
import google.generativeai as genai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure Google Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# System Tools Implementation
def open_application(app_name: str) -> str:
    """Open applications cross-platform"""
    try:
        if os.name == 'nt':  # Windows
            os.startfile(app_name)
        elif os.name == 'posix':  # macOS/Linux
            subprocess.Popen(['open' if 'darwin' in os.sys.platform else 'xdg-open', app_name])
        return f"✅ Opened {app_name}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def search_web(query: str) -> str:
    """Perform web search"""
    try:
        webbrowser.open(f"https://www.google.com/search?q={query}")
        return f"🔍 Searched for: {query}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def read_file(file_path: str) -> str:
    """Read file contents"""
    try:
        if not os.path.exists(file_path):
            return "❌ File not found"
        with open(file_path, 'r') as f:
            return f.read()[:5000]  # Limit to first 5000 chars
    except Exception as e:
        return f"❌ Error: {str(e)}"

def run_script(script_path: str) -> str:
    """Execute scripts safely"""
    try:
        # Security check - validate path
        allowed_paths = ["/scripts/", "./scripts/"]
        if not any(path in script_path for path in allowed_paths):
            return "❌ Access denied to path"
        
        result = subprocess.run(
            script_path, 
            shell=True, 
            capture_output=True, 
            text=True,
            timeout=30  # Prevent long-running scripts
        )
        return result.stdout or "🟢 Script executed"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Define the Gemini model with tools
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",  # Or another Gemini model
    tools=[
        {
            "function_declarations": [
                {
                    "name": "open_application",
                    "description": "Open applications like browsers, editors, or system apps",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "app_name": {"type": "string", "description": "Application name e.g. 'chrome', 'spotify'"}
                        },
                        "required": ["app_name"]
                    }
                },
                {
                    "name": "search_web",
                    "description": "Perform web searches",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Search keywords"}
                        },
                        "required": ["query"]
                    }
                },
                {
                    "name": "read_file",
                    "description": "Read text files",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "file_path": {"type": "string", "description": "Relative or absolute file path"}
                        },
                        "required": ["file_path"]
                    }
                },
                {
                    "name": "run_script",
                    "description": "Execute scripts or commands",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "script_path": {"type": "string", "description": "Path to script or command"}
                        },
                        "required": ["script_path"]
                    }
                }
            ]
        }
    ]
)

# Global queue for voice input
voice_queue = queue.Queue()

def listen_for_voice_input():
    """Listens for voice commands and puts them in a queue."""
    r = sr.Recognizer()
    while True:
        with sr.Microphone() as source:
            print("Say something...")
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=5)
                text = r.recognize_google(audio)
                print(f"You said: {text}")
                voice_queue.put(text)
            except sr.UnknownValueError:
                print("Could not understand audio")
            except sr.RequestError as e:
                print(f"Could not request results from Google Speech Recognition service; {e}")
            except Exception as e:
                print(f"Error during voice recognition: {e}")

# Tool Execution Handler
def execute_tool_call(tool_call):
    func_name = tool_call.function_call.name
    args = dict(tool_call.function_call.args)
    
    print(f"⚙️ Executing: {func_name}({args})")
    
    if func_name == "open_application":
        return open_application(args.get("app_name"))
    elif func_name == "search_web":
        return search_web(args.get("query"))
    elif func_name == "read_file":
        return read_file(args.get("file_path"))
    elif func_name == "run_script":
        return run_script(args.get("script_path"))
    else:
        return "⚠️ Unknown command"

# Main Chat Loop
def main():
    chat = model.start_chat(history=[])
    
    # Start the voice listener thread
    voice_thread = threading.Thread(target=listen_for_voice_input, daemon=True)
    voice_thread.start()
    
    print("🚀 System Assistant Ready (type 'exit' to quit or speak a command)")
    
    while True:
        try:
            # Check for text or voice input
            if not voice_queue.empty():
                user_input = voice_queue.get()
            else:
                user_input = input("\n👤 You: ")

            if user_input.lower() in ["exit", "quit"]:
                break
                
            response = chat.send_message(user_input)
            
            # Process potential tool calls
            while hasattr(response.candidates[0].content.parts[0], 'function_call'):
                tool_call = response.candidates[0].content.parts[0]
                
                output = execute_tool_call(tool_call)
                
                response = chat.send_message(
                    genai.types.ToolResponse(tool_name=tool_call.function_call.name, tool_state=[{"output": output}])
                )
            
            # Print final text response
            print(f"\n🤖 Assistant: {response.text}")
        
        except Exception as e:
            print(f"⚠️ Error: {str(e)}")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'pyaudio'

In [7]:
import os
from openai import OpenAI
import asyncio
from agents import Agent, Runner
from dotenv import load_dotenv

In [8]:
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))